In [30]:
import random
import time
import pynq
from pynq import Overlay
import numpy as np
from pynq import Clocks
from pynq import allocate
import sys
import struct
import matplotlib.pyplot as plt

#software function
def LDL_dsolve(n, X, D, Xout_sw):
    for i in range(n):
        Xout_sw[i] = X[i] / D[i]

def generate_random_array(n, min_value, max_value):
    return [random.uniform(min_value, max_value) for _ in range(n)]

def round_array(arr, decimals):
    return [round(num, decimals) for num in arr]

def measure_execution_time(func, *args):
    start_time = time.time()
    func(*args)
    end_time = time.time()
    return end_time - start_time

def compare_arrays(arr1, arr2, tolerance=1e-7):
    if len(arr1) != len(arr2):
        return 0
    
    for a, b in zip(arr1, arr2):
        if abs(a - b) > tolerance:
            return 0
    
    return 1

In [31]:
padding = False
n = 8
# generate and round X and D
X_sw = generate_random_array(n, 0.5, 500.0)
X_sw = round_array(X_sw, 7)

D_sw = generate_random_array(n, 0.5, 500.0)
D_sw = round_array(D_sw, 7)

Xout_sw = [0.0] * n

In [32]:
print(measure_execution_time(LDL_dsolve, n, X_sw, D_sw, Xout_sw)) #printing software execution time

2.5272369384765625e-05


In [33]:
alveare = Overlay("./bitstream/doublerate.bit")
# alveare?

padding = False

if(n%2 == 1): #if the length of the arrays is odd then we need to make them of an even length
    padding = True
    n += 1


ape = alveare.ldl_dsolve_0

X_hw = pynq.allocate(n, np.float64)
D_hw = pynq.allocate(n, np.float64)
Xout_hw = pynq.allocate(n, np.uint64)

if(padding):
    X_sw.append(1.0)
    D_sw.append(1.0)
X_hw[:] = X_sw
D_hw[:] = D_sw




#communicate the buffer address
ape.register_map

ape.write(0x18, X_hw.physical_address)
ape.write(0x24, D_hw.physical_address)
ape.write(0x30, Xout_hw.physical_address)
ape.write(0x10, n)

# ape.read(0x10)
# ape.read(0x18)
# ape.read(0x24)
# ape.read(0x30)

In [34]:
start_time_hw = time.time()
#start
ape.write(0x00, 0x01)
#polling
while ape.read(0x00) & 0x4 != 4:
    pass
#read back
Xout_hw.sync_from_device()
end_time_hw = time.time()
print(end_time_hw - start_time_hw) #printing hardware execution time

0.0020067691802978516


In [35]:
Xfinal = []
if(padding): #remove the final padding so that it is not printing the added element
    n -= 1

for i in range(n):
    uint64_bytes = struct.pack('Q', Xout_hw[i])
    # Unpack bytes as double
    double_value = struct.unpack('d', uint64_bytes)[0]
    Xfinal.append(double_value)
Xfinal

[0.24957905108832443,
 1.5230178748141003,
 0.8981124305305785,
 0.9160646179635212,
 1.3506632732003265,
 1.4434591281858162,
 1.1821135725410672,
 2.615789726083828]

In [36]:
if(compare_arrays(Xfinal, Xout_sw) == 1): 
    print("the two arrays are the same")
else:
    print("the two arrays are different")

the two arrays are the same
